<a href="https://colab.research.google.com/github/feiduobaby/good-first-issue/blob/main/Homework3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **3. Machine Learning for Classification**


3.1 Data Preparation